# Data Wrangling vom Projekt Climate Change 
Wir haben von Meteoswiss Gitternetzdaten für die Schweiz für den Zeitraum 1961–2020 erhalten. Wir haben zwei Ordner erhalten, welche pro Jahr ein File haben. Diese Files haben die Messwerte pro Monat für die Schweiz mit Auflösung von einem 2 km Raster gespeichert. Nachfolgend wird gezeigt wie man aus diesem Format die Werte für eine spezifische Koordinate erhält.

In [ ]:
import pandas as pd
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import time

from netCDF4 import Dataset

Dieser Teil holt die aktuellen Koordinaten bezüglich der iploaction.

In [ ]:
#find your ip adress and the geocoordination from it
from bs4 import BeautifulSoup
import requests

page = requests.get("https://iplocation.com/")
soup = BeautifulSoup(page.content, 'html.parser')

ip_adress = soup.find("b").get_text()
lattitude = soup.find("td", {"class": "lat"}).get_text()
longitude = soup.find("td", {"class": "lng"}).get_text()
print("your ip adress:", ip_adress)
print("latitude:", lattitude)
print("longitude:", longitude)

Für die Funktion brauchen wir die Koordinaten im float Datentyp.

In [ ]:
#instead of latitude and longitude of your ip you can also insert one manually
lat_location = float(lattitude) #45.897970 tessin zum testen      
lon_location = float(longitude) #8.986362  tessin zum testen            

Wir haben zwei Funktionen, eine für den Niederschlag und eine für die Temperatur erstellt. Die Funktion unterscheiden sich nur im Pfad auf welche Files sie zugreifen. Die Funktion gibt für ein definiertes Jahr alle Messwerte pro Monat als eine Liste zurück. Der Input min_index_lon und min_index_lat müssen zuerst berechnet werden. Diese Variablen werden verwendet, damit am richtigen Ort des zweidimensionalen Arrays die Werte herausgelesen werden.

In [ ]:
def find_precipitation(year,min_index_lon,min_index_lat):

    #open file
    path = "RhiresM_1961_2019_ch01r.swisscors/RhiresM_ch01r.swisscors_{Y}01010000_{Y}12010000.nc".format(Y=year)
    meteoswiss_data = Dataset(path)

    #creating a List and adding the Value of the same messurment point to it
    
    timeseries = []
    precipitation = meteoswiss_data.variables['RhiresM']

    for month in range(0,12):
        timeseries.append(precipitation[month][min_index_lon][min_index_lat])
    return(timeseries)

    #close netcdf
    meteoswiss_data.close()


In [ ]:
def find_temperatur(year,min_index_lon,min_index_lat):


    #open file
    path = "TabsM_1961_2017_ch01r.swisscors/TabsM_ch01r.swisscors_{Y}01010000_{Y}12010000.nc".format(Y=year)
    meteoswiss_data = Dataset(path)

    #creating a List and adding the Value of the same messurment point to it
    
    timeseries = []
    precipitation = meteoswiss_data.variables['TabsM']

    for month in range(0,12):
        timeseries.append(precipitation[month][min_index_lat][min_index_lon])
    return(timeseries)

    #close netcdf
    meteoswiss_data.close()

Die Nächsten zwei Codeabschnitte sind fast identisch. Sie unterscheiden sich darin das der einte Teil für den Niederschlag und der andere für die Temperatur ist. Die Netcdf Dateien bestehen aus einem 3 Dimensionalen Array wobei die einte Dimension die Zeit und die anderen zwei Dimensionen die Latitude und die Longitude darstellen. Die Dimension Zeit wird mit einer "For Loop" und der zuvor erstellten Funktion ermittelt. 
Das Gitternetz hat in der horizontalen und vertikalen Achse eine Auflösung von 2km. Für jeden Schnittpunkt dieser Linie gibt es einen Messpunkt und ein dazu korrespondierendes Koordinatenpaar. Um für eine belibige Koordinate den richtigen Schnittpunkt(Messpunkt) zu finden subtrahieren wir die eingegeben Koordinaten von allen Messpunkten. Um beim Vergleich keine negativen Zahlen zu haben, quadrieren wir anschliessend diese Werte. Danach können wir mit einer Funktion den Index herauslesen, der am nächsten bei null also bei einem Messpunkt ist. Dieser Index zeigt, wo im zweidimensionalen Array der gesuchte Wert ist.

In [ ]:
starttime = time.time()
path = "RhiresM_1961_2019_ch01r.swisscors/RhiresM_ch01r.swisscors_196101010000_196112010000.nc"
meteoswiss_data = Dataset(path)

# read latitutde and longitude variable
lat = meteoswiss_data.variables['lat'][:] 
lon = meteoswiss_data.variables['lon'][:]

#calculate the min distance of a measurement station
sq_distance = ((lat_location-lat)**2)+((lon_location-lon)**2)
min_sq_distance = sq_distance.min()

#find the index of the array with the smallest distance
index = ma.where(sq_distance == sq_distance.min())
min_index_lat = index[0][0]
min_index_lon = index[1][0]

#extract the precipitation for all years and saving them to a list
precipitation_year = []
for year in range(1961,2020):
    precipitation_year = precipitation_year + find_precipitation(year,min_index_lat,min_index_lon)

#creating a panda with the precipitation for the city
data_range = pd.date_range(start = '1961', end= '2020', freq='M')
timeseries_precipitation = pd.DataFrame(precipitation_year, columns = ['precipitation'], index = data_range)
print(timeseries_precipitation)
print("Time:", time.time()-starttime,"sekunden")

In [ ]:
starttime = time.time()
path = "TabsM_1961_2017_ch01r.swisscors/TabsM_ch01r.swisscors_196101010000_196112010000.nc"
meteoswiss_data = Dataset(path)

# read latitutde and longitude variable
lat = meteoswiss_data.variables['lat'][:] 
lon = meteoswiss_data.variables['lon'][:]

#calculate the min distance of a measurement station
sq_distance = ((lat_location-lat)**2)+((lon_location-lon)**2)
min_sq_distance = sq_distance.min()

#find the index of the array with the smallest distance
index = ma.where(sq_distance == sq_distance.min())
min_index_lat = index[0][0]
min_index_lon = index[1][0]

#extract the precipitation for all years and saving them to a list
temperatur_year = []

for year in range(1961,2020):
    temperatur_year = temperatur_year + find_temperatur(year,min_index_lon,min_index_lat)

#creating a panda with the precipitation for the city
data_range = pd.date_range(start = '1961', end= '2020', freq='M')
timeseries_temp = pd.DataFrame(temperatur_year, columns = ['temperatur'], index = data_range)
print(timeseries_temp)
print("Time:", time.time()-starttime,"sekunden")

In [ ]:
import plotly.express as px
import statsmodels
import plotly.graph_objects as go

df = timeseries_temp

fig=go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['temperatur'], mode='markers', name="Alle Messwerte"))

# Define Text for Hover
fig.update_traces(hovertemplate='Jahr: %{x} <br>Temperatur: %{y}\u00B0C')

# plotly figure layout
fig.update_layout(template = 'seaborn', 
                title="Monatliche Durchschnittstemperatur", 
                xaxis_title = 'Zeit in Monaten', yaxis_title = 'Temperatur in \u00B0C')

fig.show()


In [ ]:
df = timeseries_precipitation

fig=go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['precipitation'], mode='markers', name="Alle Messwerte"))

#Adjust Hover Text
fig.update_traces(hovertemplate='Jahr: %{x} <br>Niederschlag: %{y}mm')

# plotly figure layout
fig.update_layout(template = 'seaborn', 
                title="Monatlicher Niederschlag", 
                xaxis_title = 'Zeit in Monaten', yaxis_title = 'Niederschlag in mm')

fig.show()